In [21]:
import sys

In [22]:
'geopandas' in sys.modules

False

In [26]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


'C:\Program' is not recognized as an internal or external command,
operable program or batch file.


In [24]:
import requests
import json
import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")

ModuleNotFoundError: No module named 'geopandas'

In [10]:
conda create -n geo_env
conda activate geo_env
conda config --env --add channels conda-forge
conda config --env --set channel_priority strict
conda install python=3 geopandas

SyntaxError: invalid syntax (<ipython-input-10-df2add3ff41b>, line 1)

In [ ]:
# load AOI
with open(r'./Data/AmaBoundary.geojson') as inf:
    regions_dict = json.load(inf)

In [ ]:
# download market 

URL = 'https://api.ohsome.org/v1/elements/geometry'
data = {
    "bpolys": json.dumps(regions_dict),
    "time": "2022-01-01",
    "filter": "amenity=marketplace"
}
response = requests.post(URL, data=data)
display(response)

# download data
output = "./output/market.geojson"
with open(output, "w") as f:
    json.dump(response.json(), f)

In [ ]:
df = gpd.read_file(output)
df.head()
df.plot()

In [ ]:
# download highway

URL = 'https://api.ohsome.org/v1/elements/geometry'
data = {
    "bpolys": json.dumps(regions_dict),
    "time": "2022-01-01",
    "filter": "highway=primary or highway=secondary"
}
response = requests.post(URL, data=data)
display(response)

# download data
output = "./output/highway.geojson"
with open(output, "w") as f:
    json.dump(response.json(), f)

In [ ]:
df = gpd.read_file(output)
df.head()
df.plot()

In [ ]:
# download industry 

URL = 'https://api.ohsome.org/v1/elements/geometry'
data = {
    "bpolys": json.dumps(regions_dict),
    "time": "2022-01-01",
    "filter": "landuse = industrial"
}
response = requests.post(URL, data=data)
display(response)

# download data
output = "./output/industry.geojson"
with open(output, "w") as f:
    json.dump(response.json(), f)

In [ ]:
df = gpd.read_file(output)
df.head()
df.plot()

# Computing Euclidean Distance

In [ ]:
import rasterio as rio 

acc_image = rio.open('./data/acc_img_bgrn.tif', mode="r")
  
# load AOI
AOI = fiona.open(r'./Data/AmaBoundary.geojson', 'r') 

In [ ]:
import fiona
import distancerasters as dr
from glob import glob

filelist = glob("./output/*.geojson")
print(filelist)
for file in filelist:
    shp = fiona.open(file, "r")

    # resolution (in units matching projection) at which vector data will be rasterized
    pixel_size =acc_image.res[0]

    # create binary raster and affine
    rv_array, affine = dr.rasterize(shp, 
                pixel_size= pixel_size, 
                bounds=AOI.bounds 
                )

    # generate distance array and output to geotiff
    filename = []
    for file in filelist:
      head, tail = os.path.split(file)
      col_names=tail[:-8]
      filename.append(col_names)

    for output in filename:
    my_dr = dr.DistanceRaster(rv_array,
                              affine=affine,
                              output_path=f"output/{output}.tif")